# motive
1. use private gpt and introduction to tree index
## notebook work plan
1. prepare env and define model
2. read doc and metadata extractor
3. embeding vector and tree
4. build query agent
5. Give hint to patient, what next info he need for particular drug or disease


In [ ]:
# !pip install -q pygpt4all
!pip install -q llama-index
!pip install -q sentence_transformers
!pip install -q transformers
!pip install -q einops
!pip install -q accelerate
!pip install -q pypdf
!pip install -q langchain

# importing lib

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index import (
    GPTVectorStoreIndex,
    LangchainEmbedding,
    LLMPredictor,
    ServiceContext,
    StorageContext,
    download_loader,
    PromptHelper,
     SimpleDirectoryReader,
    VectorStoreIndex,
     SummaryIndex,
    SimpleKeywordTableIndex,
    ServiceContext,
)
from llama_index.schema import IndexNode
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.llms import OpenAI

# loading data

In [ ]:
# load documents
documents = SimpleDirectoryReader("./data").load_data()

In [ ]:
len(documents)

23

# loading embed model

In [ ]:
# use Huggingface embeddings
from llama_index.embeddings import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

# loading LLM model

In [ ]:
!pip install -q llama-cpp-python
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

model="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)
service_context = ServiceContext.from_defaults(embed_model=embed_model,llm=llm)

total size (MB): 7365.83


7025it [00:34, 201.08it/s]                          
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor   

# metadata extraction

In [ ]:
# !pip install -q chromadb
# import chromadb
# from llama_index.vector_stores import ChromaVectorStore
# from llama_index import StorageContext

# chroma_client = chromadb.PersistentClient()
# chroma_collection = chroma_client.get_or_create_collection("chroma_Tymlos")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
# load from disk
# db2 = chromadb.PersistentClient(path="./chroma")
# chroma_collection = db2.get_or_create_collection("chroma_Tymlos")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# # index = VectorStoreIndex.from_vector_store(
# #     vector_store,
# #     service_context=service_context,
# # )

In [ ]:
!pip install -q span-marker

## define metadata type

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index import load_index_from_storage, StorageContext
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    EntityExtractor,
)

from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

metadata_extractor = MetadataExtractor(
    extractors=[
        TitleExtractor(nodes=2,llm=llm),
        # QuestionsAnsweredExtractor(questions=3),
        #  SummaryExtractor(summaries=["prev", "self"],llm=llm),
        # KeywordExtractor(keywords=5,llm=llm),
        # EntityExtractor(prediction_threshold=0.5),
    ],
)

node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
)


In [ ]:
docs = documents[:6]


In [ ]:
len(docs)

6

## node parsing

In [ ]:
# assume documents are defined -> extract nodes
nodes = node_parser.get_nodes_from_documents(docs)


llama_print_timings:        load time = 224255.85 ms
llama_print_timings:      sample time =   232.55 ms /   158 runs   (    1.47 ms per token,   679.42 tokens per second)
llama_print_timings: prompt eval time = 237230.84 ms /   533 tokens (  445.09 ms per token,     2.25 tokens per second)
llama_print_timings:        eval time = 93344.11 ms /   157 runs   (  594.55 ms per token,     1.68 tokens per second)
llama_print_timings:       total time = 331388.06 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time = 224255.85 ms
llama_print_timings:      sample time =   157.85 ms /   134 runs   (    1.18 ms per token,   848.91 tokens per second)
llama_print_timings: prompt eval time = 230275.43 ms /   461 tokens (  499.51 ms per token,     2.00 tokens per second)
llama_print_timings:        eval time = 77463.59 ms /   133 runs   (  582.43 ms per token,     1.72 tokens per second)
llama_print_timings:       total time = 308300.51 ms
Llama.generate: prefix-match hit

lla

In [ ]:
#printing for page [:3]
nodes

[TextNode(id_='0685f339-b7b5-448d-b4f0-8fa1e24c3aae', embedding=None, metadata={'page_label': '1', 'file_name': 'Tymlos.pdf', 'document_title': 'Based on the unique entities, titles, and themes provided, a comprehensive title for this document could be:\n\n"Tymlos (Abaloparatide) Injection: Side Effects, Usage Information, and How to Deal with Dizziness"\n\nThis title accurately reflects the content of the document, which includes information about Tymlos injection, its side effects, usage instructions, and how to deal with dizziness. It also highlights the unique entity of Tymlos and its connection to osteoporosis treatment.'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='16973005-bba2-45c3-b4e7-557886e00d70', node_type=None, metadata={'page_label': '1', 'file_name': 'Tymlos.pdf'}, hash='07bcf6ebae0205104f584328cedeab25c30b7d329dc022a7ad2b70162d8a1c04')}, hash='07bcf6ebae0205104f584328cedeab25c3

In [ ]:

# docs.storage_context.persist(persist_dir="./chroma")

In [ ]:
# from llama_index import StorageContext, load_index_from_storage

# storage_context = StorageContext.from_defaults(persist_dir="./chroma")
# index = load_index_from_storage(storage_context=storage_context)

In [ ]:
service_context= ServiceContext.from_defaults(
    node_parser= node_parser,
    # storage_context= storage_context,
)

******
Could not load OpenAI model. Using default LlamaCPP=llama2-13b-chat. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******


llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_0     [  5120

******
Could not load OpenAIEmbedding. Using HuggingFaceBgeEmbeddings with model_name=BAAI/bge-small-en. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******


# with simple vectore store
* for comparing result(performance & latency) with tree indexing

In [ ]:
index = VectorStoreIndex(nodes, service_context=service_context)

In [ ]:
index.storage_context.persist(persist_dir=f"./chroma_data/tymlos")

In [ ]:
# set up query engine
query_engine = index.as_query_engine()
response = query_engine.query("What is tymlos?")
print(response)

  Based on the provided context information, Tymlos is a brand-name prescription medication used to treat osteoporosis after menopause. It contains the active drug abaloparatide and works by increasing bone density and strength. Tymlos comes as a solution inside a prefilled pen with 30 doses and is typically given once each day as a subcutaneous injection. Side effects of Tymlos may include dizziness, increased calcium levels, and others. It is important to follow the usage instructions provided by your healthcare provider to minimize the risk of side effects.



llama_print_timings:        load time = 253585.60 ms
llama_print_timings:      sample time =   172.70 ms /   130 runs   (    1.33 ms per token,   752.73 tokens per second)
llama_print_timings: prompt eval time = 515006.35 ms /  1028 tokens (  500.98 ms per token,     2.00 tokens per second)
llama_print_timings:        eval time = 78662.37 ms /   129 runs   (  609.79 ms per token,     1.64 tokens per second)
llama_print_timings:       total time = 594284.11 ms


In [ ]:
response = llm.complete("a poem about hills and see?")
print(response.text)

Llama.generate: prefix-match hit


  Sure! Here's a short poem about hills and sea:

Hills rise up to the sky,
Their peaks touching the clouds so high.
The sea below, so blue and wide,
Meets the shore with a gentle tide.

The hills and sea, a perfect pair,
One brings life, the other air.
Together they create a scene,
So peaceful, serene, and green.



llama_print_timings:        load time = 224255.85 ms
llama_print_timings:      sample time =   102.79 ms /    97 runs   (    1.06 ms per token,   943.68 tokens per second)
llama_print_timings: prompt eval time =  6301.97 ms /    11 tokens (  572.91 ms per token,     1.75 tokens per second)
llama_print_timings:        eval time = 53387.15 ms /    96 runs   (  556.12 ms per token,     1.80 tokens per second)
llama_print_timings:       total time = 60066.54 ms


In [ ]:
# set up query engine
query_engine = index.as_query_engine()
response = query_engine.query("tell me about mild side effect?")
print(response)

Llama.generate: prefix-match hit


  Based on the provided context information, here is the information about mild side effects of Tymlos (Abaloparatide) Injection:

Mild side effects of Tymlos may include:

* Dizziness
* Headache
* Fatigue (lack of energy)
* Pain in the upper belly
* Vertigo (problems with balance)

These side effects may go away within a few days or a couple of weeks. If they become more severe or don't go away, talk with your doctor or pharmacist. To learn about other mild side effects, talk with your doctor or pharmacist, or visit the Tymlos Medication Guide.

It is important to note that serious side effects from Tymlos aren't common, but they can occur. Call your doctor right away if you have serious side effects. Call 911 or your local emergency number if your symptoms feel life-threatening or if you think you're having a medical emergency.

Please keep in mind that this information is based on the provided context, and it is always best to consult with a medical professional for personalized



llama_print_timings:        load time = 253585.60 ms
llama_print_timings:      sample time =   300.80 ms /   256 runs   (    1.17 ms per token,   851.07 tokens per second)
llama_print_timings: prompt eval time = 623852.39 ms /  1227 tokens (  508.44 ms per token,     1.97 tokens per second)
llama_print_timings:        eval time = 163126.89 ms /   255 runs   (  639.71 ms per token,     1.56 tokens per second)
llama_print_timings:       total time = 788103.50 ms


In [ ]:
# set up query engine
query_engine = index.as_query_engine()
response = query_engine.query("can you tell me about cautions in proper list?")
print(response)

# from database

In [ ]:
from llama_index import GPTVectorStoreIndex, StorageContext, load_index_from_storage

storage_context= StorageContext.from_defaults(persist_dir=f"./chroma_data/tymlos")


In [ ]:
# load index
index_database = load_index_from_storage(storage_context)

******
Could not load OpenAI model. Using default LlamaCPP=llama2-13b-chat. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******


llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_0     [  5120

******
Could not load OpenAIEmbedding. Using HuggingFaceBgeEmbeddings with model_name=BAAI/bge-small-en. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******


In [ ]:
query_engine_database = index_database.as_query_engine()

In [ ]:
response = query_engine_database.query("give me tymlos cautions in proper list?")
print(response)

  Sure! Based on the provided context information, here are some possible cautions for Tymlos (abaloparatide) injection:

1. Dizziness: Tymlos can cause dizziness, so it's important to be careful when standing up or changing positions.
2. Increased calcium levels: Tymlos can increase calcium levels in the blood and urine, which may be a side effect of the drug.
3. Allergic reactions: Some people may be allergic to Tymlos and experience an allergic reaction when taking the drug. If you experience symptoms such as hives, difficulty breathing, or swelling of the face, lips, tongue, or throat, seek medical attention immediately.
4. Injection site reactions: Tymlos is administered via injection, and some people may experience injection site reactions such as redness, swelling, or pain at the injection site.
5. Hypokalemia (low potassium levels): Tymlos can cause low potassium levels, which can lead to muscle weakness, fatigue, and other symptoms. If you experience these symptoms, talk



llama_print_timings:        load time = 251330.88 ms
llama_print_timings:      sample time =   316.48 ms /   256 runs   (    1.24 ms per token,   808.90 tokens per second)
llama_print_timings: prompt eval time = 511480.53 ms /  1026 tokens (  498.52 ms per token,     2.01 tokens per second)
llama_print_timings:        eval time = 161297.15 ms /   255 runs   (  632.54 ms per token,     1.58 tokens per second)
llama_print_timings:       total time = 673962.34 ms


# With Tree index

In [ ]:
from llama_index import TreeIndex, GPTTreeIndex

In [ ]:
index_tree = TreeIndex(nodes,service_context=service_context)
# index_tree=TreeIndex.from_documents(docs,service_context=service_context)

In [ ]:
index_tree

In [ ]:
query_engine_tree = index_tree.as_query_engine()
response = query_engine_tree.query("what is tymlos?")
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time = 253585.60 ms
llama_print_timings:      sample time =   116.05 ms /   121 runs   (    0.96 ms per token,  1042.67 tokens per second)
llama_print_timings: prompt eval time = 707177.03 ms /  1783 tokens (  396.62 ms per token,     2.52 tokens per second)
llama_print_timings:        eval time = 48995.67 ms /   120 runs   (  408.30 ms per token,     2.45 tokens per second)
llama_print_timings:       total time = 756617.36 ms
Llama.generate: prefix-match hit


  Based on the provided context information, Tymlos (abaloparatide) is a brand-name prescription medication used to treat osteoporosis after menopause. It is given as a subcutaneous injection once daily and contains a synthetic form of the human parathyroid hormone that helps increase bone density and strength.



llama_print_timings:        load time = 253585.60 ms
llama_print_timings:      sample time =    74.53 ms /    78 runs   (    0.96 ms per token,  1046.57 tokens per second)
llama_print_timings: prompt eval time = 174723.23 ms /   641 tokens (  272.58 ms per token,     3.67 tokens per second)
llama_print_timings:        eval time = 28161.12 ms /    77 runs   (  365.73 ms per token,     2.73 tokens per second)
llama_print_timings:       total time = 203163.68 ms


In [ ]:
#this is tree index engine
response = query_engine_tree.query("what question can ask from this documents?")
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time = 253585.60 ms
llama_print_timings:      sample time =   120.42 ms /   127 runs   (    0.95 ms per token,  1054.62 tokens per second)
llama_print_timings: prompt eval time = 490071.32 ms /  1736 tokens (  282.30 ms per token,     3.54 tokens per second)
llama_print_timings:        eval time = 50532.24 ms /   126 runs   (  401.05 ms per token,     2.49 tokens per second)
llama_print_timings:       total time = 541063.31 ms
Llama.generate: prefix-match hit


  Based on the provided context information, here are some potential questions that could be asked based on the content of the document:

1. What are the common side effects of Tymlos injection?
2. How can I deal with dizziness and orthostatic hypotension (low blood pressure) caused by Tymlos?
3. What are the serious side effects of Tymlos, and how can I identify them?
4. Can I take Tymlos if I have a history of allergic reactions or kidney problems?
5. How long do the side effects of Tymlos usually last, and when should I seek medical attention if they persist or worsen?
6. Are there any specific precautions or warnings that I should be aware of when taking Tymlos?
7. Can Tymlos be taken with other medications, and if so, what are the potential interactions I should be aware of?
8. What is the proper dosage and administration of Tymlos, and how should I use it to maximize its effectiveness and minimize side effects?

These questions can help individuals who are taking Tymlos or consid


llama_print_timings:        load time = 253585.60 ms
llama_print_timings:      sample time =   243.00 ms /   256 runs   (    0.95 ms per token,  1053.48 tokens per second)
llama_print_timings: prompt eval time = 179273.25 ms /   656 tokens (  273.28 ms per token,     3.66 tokens per second)
llama_print_timings:        eval time = 94812.83 ms /   255 runs   (  371.82 ms per token,     2.69 tokens per second)
llama_print_timings:       total time = 275041.94 ms


In [ ]:
#this is vector index engine
response = query_engine.query("what question can ask from this documents?")
print(response)

Llama.generate: prefix-match hit


  Based on the provided context information, here are some possible questions that could be asked based on the content of the documents:

1. What are the potential side effects of using Tymlos (Abaloparatide) injection?
2. How can one deal with dizziness and orthostatic hypotension while taking Tymlos?
3. Are there any specific instructions for usage and administration of Tymlos?
4. Can Tymlos cause increased calcium levels in the blood or urine?
5. Are there any recommendations for preventing or reducing the risk of side effects associated with Tymlos?
6. What is the unit of measurement used to describe blood pressure, and how can one monitor their blood pressure while taking Tymlos?



llama_print_timings:        load time = 251330.88 ms
llama_print_timings:      sample time =   159.74 ms /   164 runs   (    0.97 ms per token,  1026.69 tokens per second)
llama_print_timings: prompt eval time = 152090.43 ms /   551 tokens (  276.03 ms per token,     3.62 tokens per second)
llama_print_timings:        eval time = 61156.77 ms /   163 runs   (  375.19 ms per token,     2.67 tokens per second)
llama_print_timings:       total time = 213865.10 ms


In [ ]:
#this is tree index engine
response = query_engine_tree.query("summary of this disease and medicine and overdose?")
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time = 253585.60 ms
llama_print_timings:      sample time =   119.23 ms /   126 runs   (    0.95 ms per token,  1056.82 tokens per second)
llama_print_timings: prompt eval time = 489822.43 ms /  1739 tokens (  281.67 ms per token,     3.55 tokens per second)
llama_print_timings:        eval time = 50021.57 ms /   125 runs   (  400.17 ms per token,     2.50 tokens per second)
llama_print_timings:       total time = 540303.32 ms
Llama.generate: prefix-match hit


  Based on the provided context information, here is a summary of the disease, medicine, and overdose:

Disease: Osteoporosis

Osteoporosis is a condition where the bones become weak and brittle, leading to an increased risk of fractures. It is often referred to as a "silent disease" because it can progress without any noticeable symptoms until a fracture occurs.

Medicine: Tymlos (Abaloparatide) Injection

Tymlos is a medication used to treat osteoporosis in postmenopausal women. It is given as an injection and works by increasing bone density, which can help reduce the risk of fractures. Common side effects of Tymlos include nausea, headache, fatigue, pain in the upper belly, and vertigo (problems with balance).

Overdose:

If you suspect an overdose of Tymlos has occurred, it is important to seek medical attention immediately. Signs of overdose may include allergic reactions, dizziness, and orthostatic hypotension (low blood



llama_print_timings:        load time = 253585.60 ms
llama_print_timings:      sample time =   245.23 ms /   256 runs   (    0.96 ms per token,  1043.93 tokens per second)
llama_print_timings: prompt eval time = 179181.98 ms /   659 tokens (  271.90 ms per token,     3.68 tokens per second)
llama_print_timings:        eval time = 94339.00 ms /   255 runs   (  369.96 ms per token,     2.70 tokens per second)
llama_print_timings:       total time = 274487.77 ms


In [ ]:
#this is vector index engine
response = query_engine.query("summary of this disease and medicine and overdose?")
print(response)

Llama.generate: prefix-match hit


  Based on the provided context information, here is a summary of the document's content:

The document discusses Tymlos (abaloparatide) injection, a medication used to treat osteoporosis in postmenopausal women. It provides information on the side effects of the injection, including redness, swelling, and pain at the injection site, as well as dizziness and orthostatic hypotension. The document also explains how to deal with these side effects and provides instructions for proper usage. Additionally, it highlights the unique entity of Tymlos and its connection to osteoporosis treatment.

There is no mention of overdose in the provided context information. Therefore, I cannot provide any information on overdose. However, if you have any other questions or concerns about Tymlos or osteoporosis treatment, I would be happy to help as best I can based on my training and knowledge.



llama_print_timings:        load time = 251330.88 ms
llama_print_timings:      sample time =   196.37 ms /   205 runs   (    0.96 ms per token,  1043.93 tokens per second)
llama_print_timings: prompt eval time = 212265.75 ms /   775 tokens (  273.89 ms per token,     3.65 tokens per second)
llama_print_timings:        eval time = 77745.57 ms /   204 runs   (  381.11 ms per token,     2.62 tokens per second)
llama_print_timings:       total time = 290778.82 ms


# storing tree index

In [ ]:
index_tree.storage_context.persist(persist_dir=f"./chroma_tree/tymlos")

In [ ]:
storage_tree= StorageContext.from_defaults(persist_dir=f"./chroma_tree/tymlos")
index_tree_database = load_index_from_storage(storage_tree)
query_engine_tree_database = index_tree_database.as_query_engine()


******
Could not load OpenAI model. Using default LlamaCPP=llama2-13b-chat. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******


llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_0     [  5120

******
Could not load OpenAIEmbedding. Using HuggingFaceBgeEmbeddings with model_name=BAAI/bge-small-en. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******


In [ ]:
response = query_engine_tree_database.query("give me tymlos cautions in proper list and document summary?")
print(response)


llama_print_timings:        load time = 139906.68 ms
llama_print_timings:      sample time =   117.58 ms /   124 runs   (    0.95 ms per token,  1054.57 tokens per second)
llama_print_timings: prompt eval time = 506035.48 ms /  1806 tokens (  280.20 ms per token,     3.57 tokens per second)
llama_print_timings:        eval time = 48634.87 ms /   123 runs   (  395.41 ms per token,     2.53 tokens per second)
llama_print_timings:       total time = 555116.65 ms
Llama.generate: prefix-match hit


  Based on the provided context information, here are the Tymlos cautions in a proper list format:

1. Allergic reaction: If you experience symptoms such as hives, difficulty breathing, or swelling of the face, lips, tongue, or throat while taking Tymlos, seek medical attention immediately.
2. Dizziness and orthostatic hypotension (low blood pressure when standing up quickly): These side effects can occur especially when you first start taking Tymlos. To minimize the risk, rise slowly from a sitting or lying position, and avoid alcohol and activities that may cause dizziness.
3. Increased calcium levels in your blood or urine: Your doctor may monitor your calcium levels during treatment with Tymlos. If you experience symptoms such as nausea, vomiting, constipation, or frequent urination, talk to your doctor.

As for the document summary, here is a comprehensive title that accurately reflects the content of the document:

"Tymlos (Abaloparatide) Injection: Side Effects, Usage Informatio


llama_print_timings:        load time = 139906.68 ms
llama_print_timings:      sample time =   243.26 ms /   256 runs   (    0.95 ms per token,  1052.38 tokens per second)
llama_print_timings: prompt eval time = 181238.96 ms /   662 tokens (  273.77 ms per token,     3.65 tokens per second)
llama_print_timings:        eval time = 95182.63 ms /   255 runs   (  373.27 ms per token,     2.68 tokens per second)
llama_print_timings:       total time = 277370.55 ms
